In [3]:
%cd NYSE-Temporal-Graph-Construction

/teamspace/studios/this_studio/NYSE-Temporal-Graph-Construction


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [91]:
import networkx as nx
from matplotlib import pyplot as plt
from tqdm import tqdm
from os.path import join as join_path
import os
from torch.utils.data import Dataset
import kagglehub
import numpy as np
import pandas as pd
from functools import cache
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cpu


In [4]:
dataset_path = kagglehub.dataset_download("dgawlik/nyse")

In [62]:
fundamentals_df = pd.read_csv(join_path(dataset_path,"fundamentals.csv"))
df = pd.read_csv(join_path(dataset_path, "prices.csv"))
prices_split_adjusted_df = pd.read_csv(join_path(dataset_path, "prices-split-adjusted.csv"))
securities_df = pd.read_csv(join_path(dataset_path, "securities.csv"))

In [63]:
all_symbols = prices_split_adjusted_df['symbol'].unique()

@cache
def symbol_to_int(symbol: str):
    return all_symbols.tolist().index(symbol)

@cache
def int_to_symbol(idx):
    return all_symbols[idx]

print("Num of symbols:", len(all_symbols))

Num of symbols: 501


In [64]:
prices_split_adjusted_df['date'] = pd.to_datetime(prices_split_adjusted_df['date']).dt.date
dates = prices_split_adjusted_df['date'].sort_values().unique()


def any_to_date(date):
    if not isinstance(date, pd._libs.tslibs.timestamps.Timestamp):
        date = pd.to_datetime(date).date()
    return date

@cache
def date_to_int(date):
    date = any_to_date(date)
    return dates.tolist().index(date)

@cache
def int_to_date(idx):
    return dates[idx]

print(dates)

[datetime.date(2010, 1, 4) datetime.date(2010, 1, 5)
 datetime.date(2010, 1, 6) ... datetime.date(2016, 12, 28)
 datetime.date(2016, 12, 29) datetime.date(2016, 12, 30)]


In [ ]:

# class NYSEDailyDataset(Dataset):
#     def __init__(self, prices_df):
#         self.df = prices_df.copy()
        
#         self.df['date'] = self.df['date'].apply(date_to_int)
#         self.df['symbol'] = self.df['symbol'].apply(symbol_to_int)

#         self.df.sort_values(['date', 'symbol'], inplace=True)


#         print("df", self.df.head(10))

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
        
#         return self.df.iloc[idx].to_numpy().tolist()
        


# dataset = NYSEDailyDataset(prices_split_adjusted_df[:500])

df      date  symbol       open      close        low       high       volume
251     0       1  22.453504  22.389128  22.267525  22.625180    3815500.0
252     0       2   4.840000   4.770000   4.660000   4.940000    9837300.0
253     0       3  40.700001  40.380001  40.360001  41.040001    1701700.0
254     0       4  30.490000  30.572857  30.340000  30.642857  123432400.0
255     0       5  26.290001  26.629999  26.139999  26.690001    2455900.0
256     0       6  26.000339  26.129884  25.870792  26.177866   10829000.0
257     0       7  41.520000  42.070000  41.500000  42.200001    3650100.0
258     0       8  36.650002  37.090000  36.650002  37.299999    4710200.0
259     0       9  31.790001  31.670000  31.610001  32.189999    2102700.0
260     0      10  31.480000  31.469999  31.330000  31.840000    3472500.0


In [96]:
def load_data(target_df):

    df = target_df.copy()
        
    df['date'] = df['date'].apply(date_to_int)
    df['symbol'] = df['symbol'].apply(symbol_to_int)

    df.sort_values(['date', 'symbol'], inplace=True)

    batch = []
    current_date = df['date'].iloc[0]
    for i, data in df.iterrows():
        if data['date'] != current_date:
            yield np.array(batch)
            batch = []
            current_date = data['date']
        else:
            batch.append(data.to_numpy())

    if len(batch) > 0:
        yield np.array(batch)
        


In [98]:
for batch in load_data(prices_split_adjusted_df.sort_values('date').head(500)):
    print(batch.shape)

(467, 7)
(32, 7)


In [ ]:
class Attnetion(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.query = torch.nn.Linear(input_size, hidden_size)
        self.key = torch.nn.Linear(input_size, hidden_size)
        self.value = torch.nn.Linear(input_size, hidden_size)

        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        attn = torch.bmm(q, k.transpose(1, 2))
        attn = self.softmax(attn)

        out = torch.bmm(attn, v)
        return out

class HiddenRNN(torch.nn.Module):

    def __init__(self, input_size, hidden_size):
        pass

    def forward(self, x, attention):
        pass

class PredictorNet(torch.nn.Module):
    def __init__(self, input_size, att_size, hidden_size):
        super().__init__()
        self.attnetion = Attnetion(input_size, att_size)
        self.ffn = torch.nn.Sequential(
            torch.nn.Linear(att_size, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, hidden_size),
            torch.nn.ReLU()
        )
        
        self.rnn = torch.nn.RNN(input_size, hidden_size, num_layers=3, batch_first=True)

    def forward(self, x):
        
        att_out = self.attnetion(x)

        ffn_out = self.ffn(att_out)

        out, _ = self.rnn(ffn_out)

        return out


myNN = PredictorNet(input_size=6, att_size=64, hidden_size=32).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(myNN.parameters(), lr=1e-3)

for epoch in range(10):
    for i, data in enumerate(dataset):
        data = torch.tensor(data, dtype=torch.float32).to(device)

        optimizer.zero_grad()
        output = myNN(data)
        loss = criterion(output, data)